In [1]:
!pip install selenium

     ---------------------------------------- 9.9/9.9 MB 7.9 MB/s eta 0:00:00
     -------------------------------------- 400.2/400.2 kB 4.2 MB/s eta 0:00:00
     -------------------------------------- 58.3/58.3 kB 762.2 kB/s eta 0:00:00


In [21]:
#!pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/b7/f8/32d6b5aa4c4bc045fa2c4c58f88c325facc54721956c6313f0afea8ea853/pandas-2.1.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for numpy>=1.23.2 from https://files.pythonhosted.org/packages/72/b2/02770e60c4e2f7e158d923ab0dea4e9f146a2dbf267fec6d8dc61d475689/numpy-1.25.2-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/502.3 kB ? eta -:--:--
     -- ------------------------------------ 30.7/502.3 kB 1.3 MB/s eta 0:00:01
     ----------------- -------------------- 225.3/502.3 kB 3.5 MB/s eta 0:00:01
     -------------------------------------- 502.3/502.3 kB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/341.8 kB ? eta -:--:--
     ------------------------------------- 341.8/341.8 kB 10.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   -- ------------------------------------- 0

# Web Scraping

Import the libraries.

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

Open the Browser.

In [3]:
driver = webdriver.Edge()

Navigate to the event page.

In [4]:
driver.get("https://chess.com/events/2023-bullet-chess-championship-main-event/games")

Before going any further, it's necessary a scroll-down of the page to load more matches.
Then, get every element having the 'round-games-list-item-games' class.

In [5]:
game_items = driver.find_elements(By.CLASS_NAME, "round-games-list-item-games")

Now, extract all the links to the games.

In [6]:
# Create a new empty list
links = []

# For each game_item, find every elements 'a' with href attribute
for game_item in game_items:
    game_links = game_item.find_elements(by=By.TAG_NAME, value="a")

    # Add all links to the list
    for game_link in game_links:
        links.append(game_link.get_attribute("href"))

# Check if it loaded enough matches
if len(links) > 31:
    links = links[:32]
    print('Success')
else:
    print('Load more matches')

Success


## Extract informations

In [11]:
# Variables for summary Dataframe
player = []
color_generic = []
clock_value = []
clock_result = []
accuracy_score = []
game = []

# Variables for in-game specifics Dataframe
move = []
skill = []
turn = []
color = []
match = []
n = 0


for link in links[:3]:
    # Load the page
    driver.get(link)
    sleep(5)
    print(link)
    

    # Save the player ...
    if 'Nakamura_Hikaru-Carlsen_Magnus' in link:
        player.append('Hikaru Nakamura')
        player.append('Magnus Carlsen')
    elif 'Carlsen_Magnus-Nakamura_Hikaru' in link:
        player.append('Magnus Carlsen')
        player.append('Hikaru Nakamura')
    # ... and the piece colors
    color_generic.append('W')
    color_generic.append('B')

    # Saving clock-values and results
    for i in range(1, -1, -1):
        clock_value.append(driver.find_elements(By.CLASS_NAME, 'clock-value')[i].text)
        clock_result.append(driver.find_elements(By.CLASS_NAME, 'clock-result')[i].text)
    
    # Saving the accuracy
    for i in range(2):
        accuracy_score.append(driver.find_elements(By.CLASS_NAME, 'accuracy-score-value')[i].text)
    
    # Then save the game
    n += 1
    game.append(n)
    game.append(n)
    

    # Load the match-analysis tab
    driver.find_element(By.CSS_SELECTOR, "button[data-cy='game-sidebar-tab-analysis']").click()
    sleep(3)

    # Takes moves in the game analysis
    moves_driver = driver.find_elements(By.CLASS_NAME, 'move-node')
    moves = []
    for m in moves_driver:
        moves.append(m.find_element(By.CLASS_NAME, 'move-text').text)
        # Check if the move is 'bad/good' or 'normal'
        if 'customColor' in m.get_attribute('class'):
            # Get style
            style = m.get_attribute('style')
            index = style.find('#')
            skill.append(style[index:index + 7])
        else:
            skill.append('=')
    
    # Storing all moves
    move += moves

    # Saving the colors and the number of the game
    for i in range(len(moves)):
        turn.append(int(i/2) + 1)
        if i % 2 == 0:
            color.append('W')
        else:
            color.append('B')
        match.append(n)
    
    print('Game', n, 'fatto')

https://www.chess.com/events/2023-bullet-chess-championship-main-event/05-01/Nakamura_Hikaru-Carlsen_Magnus
ciaone
Game 1 fatto
https://www.chess.com/events/2023-bullet-chess-championship-main-event/05-02/Carlsen_Magnus-Nakamura_Hikaru
ciao
Game 2 fatto
https://www.chess.com/events/2023-bullet-chess-championship-main-event/05-03/Nakamura_Hikaru-Carlsen_Magnus
ciaone


KeyboardInterrupt: 

In [8]:
print(len(player))
print(len(color_generic))
print(len(clock_value))
print(len(clock_result))
print(len(accuracy_score))
print(len(game))

print(len(move))
print(len(skill))
print(len(turn))
print(len(color))
print(len(match))

0
64
64
64
64
64
3439
3439
3439
3439
3439


In [24]:
df_summary = pd.DataFrame({
    'player': player,
    'color_generic': color_generic,
    'clock_value': clock_value,
    'clock_result': clock_result,
    'accuracy_score': accuracy_score,
    'game': game
})

df_summary

,player,color_generic,clock_value,clock_result,accuracy_score,game
0,Hikaru Nakamura,W,0:30,DRAW,96.1,1
1,Magnus Carlsen,B,0:25,DRAW,96.0,1
2,Hikaru Nakamura,W,0:05.1,DRAW,80.4,2
3,Magnus Carlsen,B,0:05.1,DRAW,78.2,2
4,Hikaru Nakamura,W,0:37,WON,92.2,3
...,...,...,...,...,...,...
59,Magnus Carlsen,B,0:17.5,LOST,82.7,30
60,Hikaru Nakamura,W,0:03.5,WON,79.8,31
61,Magnus Carlsen,B,0:00,LOST,79.3,31
62,Hikaru Nakamura,W,0:09.8,LOST,92.4,32


In [27]:
df_games = pd.DataFrame({
    'move': move,
    'skill': skill,
    'turn': turn,
    'color': color,
    'match': match
})

df_games

,move,skill,turn,color,match
0,e4,=,1,W,1
1,e5,=,1,B,1
2,Nf3,=,2,W,1
3,Nc6,=,2,B,1
4,d4,=,3,W,1
...,...,...,...,...,...
3434,Qd2+,=,56,B,32
3435,Kf2,=,57,W,32
3436,c2,=,57,B,32
3437,Qc7,=,58,W,32


In [30]:
df_games.skill = df_games.skill.str.strip()

In [32]:
df_summary.to_csv('summary.csv', index=False)
df_games.to_csv('games.csv', index=False)

In [33]:
df_games.skill.value_counts()

skill
=          3058
#e6912c     141
#5c8bb0     130
#ee6b55      81
#b33430      25
#1baca6       4
Name: count, dtype: int64

In [35]:
df_games[df_games.skill == '#1baca6']

,move,skill,turn,color,match
1166,Bxh7+,#1baca6,23,W,11
1607,Nxg6,#1baca6,31,W,16
2173,Nxd5,#1baca6,18,B,21
3406,Qxe4,#1baca6,42,B,32


In [36]:
symbol = ['??', '?', 'X', '!', '!!']
meaning = ['Blunder', 'Mistake', 'Missed win', 'Good move', 'Brilliant move']
hex = ['#b33430', '#e6912c', '#ee6b55', '#5c8bb0', '#1baca6']

df_annotation = pd.DataFrame({
    'symbol': symbol,
    'meaning': meaning,
    'hex': hex
})

df_annotation

,symbol,meaning,hex
0,??,Blunder,#b33430
1,?,Mistake,#e6912c
2,X,Missed win,#ee6b55
3,!,Good move,#5c8bb0
4,!!,Brilliant move,#1baca6


In [37]:
df_annotation.to_csv('data/annotation.csv')

In [12]:
df_sistemaggio = pd.read_csv('data/summary.csv')

df_sistemaggio

,player,color_generic,clock_value,clock_result,accuracy_score,game
0,Hikaru Nakamura,W,0:30,DRAW,96.1,1
1,Magnus Carlsen,B,0:25,DRAW,96.0,1
2,Hikaru Nakamura,W,0:05.1,DRAW,80.4,2
3,Magnus Carlsen,B,0:05.1,DRAW,78.2,2
4,Hikaru Nakamura,W,0:37,WON,92.2,3
...,...,...,...,...,...,...
59,Magnus Carlsen,B,0:17.5,LOST,82.7,30
60,Hikaru Nakamura,W,0:03.5,WON,79.8,31
61,Magnus Carlsen,B,0:00,LOST,79.3,31
62,Hikaru Nakamura,W,0:09.8,LOST,92.4,32


In [14]:
p = []
for i in range(int(len(df_sistemaggio)/2)):
    if i % 2 == 1:
        p.append('Magnus Carlsen')
        p.append('Hikaru Nakamura')
    else:
        p.append('Hikaru Nakamura')
        p.append('Magnus Carlsen')

df_sistemaggio.player = p
df_sistemaggio

,player,color_generic,clock_value,clock_result,accuracy_score,game
0,Hikaru Nakamura,W,0:30,DRAW,96.1,1
1,Magnus Carlsen,B,0:25,DRAW,96.0,1
2,Magnus Carlsen,W,0:05.1,DRAW,80.4,2
3,Hikaru Nakamura,B,0:05.1,DRAW,78.2,2
4,Hikaru Nakamura,W,0:37,WON,92.2,3
...,...,...,...,...,...,...
59,Hikaru Nakamura,B,0:17.5,LOST,82.7,30
60,Hikaru Nakamura,W,0:03.5,WON,79.8,31
61,Magnus Carlsen,B,0:00,LOST,79.3,31
62,Magnus Carlsen,W,0:09.8,LOST,92.4,32


In [15]:
df_sistemaggio.to_csv('data/summary.csv')

In [21]:
df_sistemaggio[df_sistemaggio.accuracy_score == 100]

,player,color_generic,clock_value,clock_result,accuracy_score,game
6,Magnus Carlsen,W,0:04.4,WON,100.0,4
7,Hikaru Nakamura,B,0:02.3,LOST,100.0,4
12,Hikaru Nakamura,W,0:12.5,DRAW,100.0,7
13,Magnus Carlsen,B,0:20,DRAW,100.0,7
40,Hikaru Nakamura,W,0:06.0,DRAW,100.0,21
41,Magnus Carlsen,B,0:05.0,DRAW,100.0,21


In [25]:
indexes = df_sistemaggio[df_sistemaggio.accuracy_score == 100].index
df_sistemaggio['accuracy_score'].iloc[indexes[0]] = 83.0
df_sistemaggio['accuracy_score'].iloc[indexes[1]] = 76.1
df_sistemaggio['accuracy_score'].iloc[indexes[2]] = 93.4
df_sistemaggio['accuracy_score'].iloc[indexes[3]] = 94.1
df_sistemaggio['accuracy_score'].iloc[indexes[4]] = 90.8
df_sistemaggio['accuracy_score'].iloc[indexes[5]] = 91.5



IndexError: index 0 is out of bounds for axis 0 with size 0

In [27]:
df_sistemaggio.iloc[[6,7,12,13,40,41]]

,player,color_generic,clock_value,clock_result,accuracy_score,game
6,Magnus Carlsen,W,0:04.4,WON,83.0,4
7,Hikaru Nakamura,B,0:02.3,LOST,76.1,4
12,Hikaru Nakamura,W,0:12.5,DRAW,93.4,7
13,Magnus Carlsen,B,0:20,DRAW,94.1,7
40,Hikaru Nakamura,W,0:06.0,DRAW,90.8,21
41,Magnus Carlsen,B,0:05.0,DRAW,91.5,21


In [28]:
print('Average accuracy of Magnus Carlsen: ', df_sistemaggio[df_sistemaggio['player'] == 'Magnus Carlsen']['accuracy_score'].mean())
print('Average accuracy of Hikaru Nakamura: ', df_sistemaggio[df_sistemaggio['player'] == 'Hikaru Nakamura']['accuracy_score'].mean())

Average accuracy of Magnus Carlsen:  87.63125
Average accuracy of Hikaru Nakamura:  87.225
